In [1]:
import tensorflow as tf
from keras_preprocessing import sequence
from tensorflow import keras
from tensorflow.python.keras import Input
from tensorflow.python.keras.layers import Concatenate


C:\Users\Adil\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [200]:
import tensorflow as tf
from keras_preprocessing import sequence
from tensorflow import keras
from tensorflow.python.keras import Input
from tensorflow.python.keras.layers import Concatenate
 

In [201]:
import json

with open('train_.txt') as json_file:  
    train_data = json.load(json_file)

from random import shuffle
shuffle(train_data)



a=[]
for i in train_data:
    a.append(i['category'])
classes=set(a)

classes

{'BLACK VOICES', 'COMEDY', 'HEALTHY LIVING', 'SPORTS', 'WORLD NEWS'}

In [202]:
train_data=train_data[0:1000]
len(train_data)

1000

In [203]:
import pandas as pd

d = {}
for k,v in enumerate(classes):
    d[v] = k
    
k=[]
C=[]
for item in train_data:
    c=d[item['category']]
    C.append(c)
    EXP=item['headline']+item['short_description']
    k.append(EXP)

train_df = pd.DataFrame(k)
train_df['category']=C
train_df.columns=["sentence","category"]

In [204]:
train_df.head()

,sentence,category
0,Babies Given Antibiotics Are More Likely To De...,1
1,'Moonlight' Director Still Hasn't Gotten Over ...,4
2,Legionnaires' Disease Is Back In NYC With New ...,1
3,Jimmy Kimmel Launches A New Personal Crusade.....,2
4,5 Thanksgiving Recipes To Poison Your Uncle Ri...,2


In [205]:
from collections import Counter
import numpy as np

time_steps = 100

# building vocabulary from dataset
def build_vocabulary(sentence_list):
    unique_words = " ".join(sentence_list).strip().split()
    word_count = Counter(unique_words).most_common()
    vocabulary = {}
    for word, _ in word_count:
        vocabulary[word] = len(vocabulary)        

    return vocabulary

vocabulary = build_vocabulary(train_df["sentence"])

In [206]:
# Create datasets (Only take up to time_steps words for memory)
train_text = train_df['sentence'].tolist()
train_text = [' '.join(t.split()[0:time_steps]) for t in train_text]
train_text = np.array(train_text)
train_label = np.array(train_df['category'].tolist())

In [207]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_LENGTH = 200
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_text)
post_seq = tokenizer.texts_to_sequences(train_text)
post_seq_padded = pad_sequences(post_seq, maxlen=MAX_LENGTH)

In [208]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(post_seq_padded, train_label, test_size=0.05)

In [215]:
class Attention(tf.keras.Model):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(5)
 
    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        attention_weights = tf.reduce_max(attention_weights, axis=2)
        attention_weights = tf.expand_dims(attention_weights, -1)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)
 
        return context_vector, attention_weights

In [216]:
sequence_input = Input(shape=(MAX_LENGTH,), dtype='int32')
 
embedded_sequences = keras.layers.Embedding(len(vocabulary), 16, input_length=max_len)(sequence_input)

In [217]:
import os
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM
                                     (rnn_cell_size,
                                      return_sequences=True,
                                      return_state=True,
                                      recurrent_activation='relu',
                                      recurrent_initializer='glorot_uniform'), name="bi_lstm_0")(embedded_sequences)
 
lstm, forward_h, forward_c, backward_h, backward_c = tf.keras.layers.Bidirectional \
    (tf.keras.layers.LSTM
     (rnn_cell_size,
      dropout=0.2,
      return_sequences=True,
      return_state=True,
      recurrent_activation='relu',
      recurrent_initializer='glorot_uniform'))(lstm)

In [218]:
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

attention = Attention(32)
 
context_vector, attention_weights = attention(lstm,state_h)
 
output = keras.layers.Dense(5, activation='softmax')(context_vector)
 
model = keras.Model(inputs=sequence_input, outputs=output)
 
# summarize layers
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_22 (Embedding)        (None, 200, 16)      153984      input_21[0][0]                   
__________________________________________________________________________________________________
bi_lstm_0 (Bidirectional)       [(None, 200, 256), ( 148480      embedding_22[0][0]               
__________________________________________________________________________________________________
bidirectional_12 (Bidirectional [(None, 200, 256), ( 394240      bi_lstm_0[0][0]                  
                                                                 bi_lstm_0[0][1]                  
          

In [219]:
from keras.utils import to_categorical

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(X_train,
                    to_categorical(y_train),
                    epochs=5,
                    batch_size=64,
                    validation_split=.25, verbose=1)


Train on 712 samples, validate on 238 samples
Epoch 1/5
712/712 [==============================] - 77s 108ms/sample - loss: 1.6078 - acc: 0.2683 - val_loss: 1.6048 - val_acc: 0.2521
Epoch 2/5
712/712 [==============================] - 86s 121ms/sample - loss: 1.5961 - acc: 0.3146 - val_loss: 1.5857 - val_acc: 0.2521
Epoch 3/5
712/712 [==============================] - 95s 133ms/sample - loss: 1.5530 - acc: 0.3146 - val_loss: 1.5866 - val_acc: 0.2521
Epoch 4/5
712/712 [==============================] - 91s 127ms/sample - loss: 1.5408 - acc: 0.3146 - val_loss: 1.5520 - val_acc: 0.2521
Epoch 5/5
712/712 [==============================] - 92s 130ms/sample - loss: 1.5376 - acc: 0.3146 - val_loss: 1.5565 - val_acc: 0.2521
